In [ ]:
!pip install --upgrade pip setuptools
!pip install azure-storage-blob pandas

import pandas as pd
from azure.storage.blob import BlobServiceClient
import io  # Import io for BytesIO

import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO  # Ensure BytesIO is imported

Connecting to Azure

In [ ]:
# Azure Blob Storage connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=citibikestorageproj;AccountKey=psNYQilyqmQOGt89lwlc14aIoI6ttYuDuPolHTED0cNlxLCVRlJJpEjyFdaAItvvjC6pNcG3kD6l+AStRrGuPQ==;EndpointSuffix=core.windows.net"
container_name = "rawdata"  # Ensure this container exists in your Azure Blob Storage
file_name = "202407-citibike-tripdata_1.csv"  # Ensure this file exists in the container

try:
    # Connect to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

    # Download the CSV file as a DataFrame
    downloaded_blob = blob_client.download_blob().readall()
    data = pd.read_csv(BytesIO(downloaded_blob))  # Use io.BytesIO to read the blob

    print(data.head())  # Preview the loaded data

except Exception as e:
    print(f"An error occurred: {e}")

import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO

# Azure Blob Storage connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=citibikestorageproj;AccountKey=psNYQilyqmQOGt89lwlc14aIoI6ttYuDuPolHTED0cNlxLCVRlJJpEjyFdaAItvvjC6pNcG3kD6l+AStRrGuPQ==;EndpointSuffix=core.windows.net"
container_name = "rawdata"  # Your container name
file_name = "202407-citibike-tripdata_1.csv"  # Your file name

# Step 1: Connect to Azure Blob Storage and Download Data
try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

    # Check if blob exists
    if blob_client.exists():
        print("Blob exists. Downloading...")
        downloaded_blob = blob_client.download_blob().readall()
        print("Blob downloaded successfully.")
    else:
        print(f"Blob '{file_name}' not found in container '{container_name}'.")
        exit()

    # Load data into a pandas DataFrame
    try:
        data = pd.read_csv(BytesIO(downloaded_blob))
        print("Data loaded into DataFrame successfully.")
        print(data.head())
    except Exception as e:
        print(f"Error reading blob content: {e}")
        exit()
except Exception as e:
    print(f"Error downloading or reading blob: {e}")
    exit()

<ipython-input-3-cf520fb7f921>:13: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(BytesIO(downloaded_blob))  # Use io.BytesIO to read the blob


   Unnamed: 0           ride_id  rideable_type               started_at  \
0           0  A3818B07E831C033  electric_bike  2024-07-01 21:00:55.640   
1           1  40B46D3F898A6CA4  electric_bike  2024-07-09 18:28:39.551   
2           2  33683B345D08C2BC  electric_bike  2024-07-14 17:10:28.899   
3           3  B213C077CE572EBD  electric_bike  2024-07-01 11:37:06.440   
4           4  E42FF50D966A009B  electric_bike  2024-07-05 16:55:59.093   

                  ended_at      start_station_name start_station_id  \
0  2024-07-01 21:07:43.553         W 24 St & 7 Ave          6257.03   
1  2024-07-09 18:46:43.854   Park Pl & Buffalo Ave          3999.06   
2  2024-07-14 17:27:59.917         E 48 St & 5 Ave          6626.01   
3  2024-07-01 11:42:02.259  Melrose Ave & E 154 St          7918.12   
4  2024-07-05 16:59:41.250         W 24 St & 7 Ave          6257.03   

             end_station_name end_station_id  start_lat  start_lng    end_lat  \
0            W 20 St & 10 Ave        6306

<ipython-input-3-cf520fb7f921>:45: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(BytesIO(downloaded_blob))


Data loaded into DataFrame successfully.
   Unnamed: 0           ride_id  rideable_type               started_at  \
0           0  A3818B07E831C033  electric_bike  2024-07-01 21:00:55.640   
1           1  40B46D3F898A6CA4  electric_bike  2024-07-09 18:28:39.551   
2           2  33683B345D08C2BC  electric_bike  2024-07-14 17:10:28.899   
3           3  B213C077CE572EBD  electric_bike  2024-07-01 11:37:06.440   
4           4  E42FF50D966A009B  electric_bike  2024-07-05 16:55:59.093   

                  ended_at      start_station_name start_station_id  \
0  2024-07-01 21:07:43.553         W 24 St & 7 Ave          6257.03   
1  2024-07-09 18:46:43.854   Park Pl & Buffalo Ave          3999.06   
2  2024-07-14 17:27:59.917         E 48 St & 5 Ave          6626.01   
3  2024-07-01 11:42:02.259  Melrose Ave & E 154 St          7918.12   
4  2024-07-05 16:59:41.250         W 24 St & 7 Ave          6257.03   

             end_station_name end_station_id  start_lat  start_lng    end_lat  \


Perform Transformation


In [ ]:
try:
    # Unified Date Format
    data['start_time'] = pd.to_datetime(data['started_at'], errors='coerce').dt.strftime('%Y-%m-%d')
    data['end_time'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.strftime('%Y-%m-%d')

    # Splitting Dates into Multiple Units
    data['start_year'] = pd.to_datetime(data['started_at'], errors='coerce').dt.year
    data['start_quarter'] = pd.to_datetime(data['started_at'], errors='coerce').dt.quarter
    data['start_month'] = pd.to_datetime(data['started_at'], errors='coerce').dt.month
    data['start_day'] = pd.to_datetime(data['started_at'], errors='coerce').dt.day
    data['start_hour'] = pd.to_datetime(data['started_at'], errors='coerce').dt.hour

    data['end_year'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.year
    data['end_quarter'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.quarter
    data['end_month'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.month
    data['end_day'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.day
    data['end_hour'] = pd.to_datetime(data['ended_at'], errors='coerce').dt.hour

    # Removing NULL Values
    data.dropna(subset=['started_at', 'ended_at'], inplace=True)

    # Removing Duplicate Rows
    data.drop_duplicates(inplace=True)

    # Adding New Columns (Trip Duration in Minutes)
    data['trip_duration_minutes'] = (
        pd.to_datetime(data['ended_at'], errors='coerce') - pd.to_datetime(data['started_at'], errors='coerce')
    ).dt.total_seconds() / 60

    # Summing Columns (Latitude + Longitude)
    data['start_lat_lng_sum'] = data['start_lat'] + data['start_lng']
    data['end_lat_lng_sum'] = data['end_lat'] + data['end_lng']

    print("Transformations applied successfully.")
except Exception as e:
    print(f"Error during transformations: {e}")
    exit()

# Step 3: Create Data Mapping for Data Dictionary
try:
    data_mapping = pd.DataFrame({
        'Field Name': data.columns,
        'Data Type': [data[col].dtype for col in data.columns],
        'Description': ['Description for ' + col for col in data.columns],
        'Source Column': data.columns,
        'Destination Column': data.columns
    })

    # Save Transformed Data and Data Mapping
    data.to_csv('transformed_data.csv', index=False)  # Save the transformed data
    data_mapping.to_csv('data_mapping.csv', index=False)  # Save the data dictionary

    print("Transformation complete. Files saved: 'transformed_data.csv' and 'data_mapping.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

Transformations applied successfully.
Transformation complete. Files saved: 'transformed_data.csv' and 'data_mapping.csv'.


Saving and uploading Transformed Data


In [ ]:
from io import BytesIO
from azure.storage.blob import BlobServiceClient

# Save transformed data to a CSV in memory
try:
    # Create an in-memory bytes buffer
    output = BytesIO()
    data.to_csv(output, index=False, encoding='utf-8')  # Save data to the buffer
    output.seek(0)  # Reset the buffer position to the beginning

    # Connect to Azure Blob Storage
    blob_client = blob_service_client.get_blob_client(container=container_name, blob="transformed_citibike_data.csv")

    # Upload the CSV to Azure Blob Storage
    blob_client.upload_blob(output, overwrite=True)
    print("Transformed data uploaded successfully.")
except Exception as e:
    print(f"Error uploading transformed data: {e}")


Transformed data uploaded successfully.
